# Equal-Weight Index Fund

## 📌 Introduction & Library Imports
The S&P 500 is the world's most followed stock market index. Instead of using the full set of 500 companies or relying on a paid data provider, this project focuses on creating an equal-weight version of the index using a subset of 200 S&P 500 companies, retrieved with the Finnhub API.

The script calculates how many shares of each stock to purchase based on a user's total portfolio value, ensuring equal dollar allocation across the 200 selected companies.

I begin by importing the essential libraries needed for data handling and API requests.

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## 📥 Importing Our List of Stocks
For this project, a static list of 200 selected companies from the S&P 500 has been used instead of the full index. The complete list of S&P 500 constituents changes periodically, but accessing it in real time via Standard & Poor’s requires a paid subscription, which was intentionally avoided.

The selected 200 companies have been imported from a pre-prepared dataset placed in the project directory link -> http://localhost:8888/edit/starter_files/newSP.csv. This list serves as the foundation for building an equal-weight index fund simulation using live market data.

All stock information, including price and market capitalization, was retrieved using the Finnhub API, which offers reliable and free access to real-time financial data.

In [2]:
stocks =pd.read_csv('newSP.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
193,HCA
194,HD
195,HES
196,HIG


## 🔐 Acquiring an API Token
To access real-time stock data, I used the Finnhub API, which offers a generous free tier suitable for projects like this.

In this project, I stored the API token directly in a variable for quick use. However, it can also be stored securely in a separate secrets.py file to keep credentials out of version control. That file would contain a single variable holding the API key and should be placed in the same directory as this notebook.

To obtain the token, I registered at finnhub.io and used the key in the following format:

In [4]:
api_key = 'd1h9n5hr01qsvr28qjbgd1h9n5hr01qsvr28qjc0'

## 📡 Making My First API Call
To retrieve live market data, I structured my API calls using the Finnhub API.

For each stock, I needed two key pieces of information:

Market Capitalization

Current Stock Price

I made separate GET requests to Finnhub’s /quote endpoint to get the stock price, and to /stock/profile2 to fetch the market capitalization and other company details. This setup allowed me to collect the data required for portfolio allocation across the 200 selected companies.


In [5]:
symbol = 'AAP'

url = f"https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={api_key}"

url2 = f"https://finnhub.io/api/v1/quote?symbol={symbol}&token={api_key}"

data = requests.get(url).json()
data2 = requests.get(url2).json()

## 🧮 Parsing My API Call
The API responses provided all the information I needed to build the equal-weight S&P 500 strategy. However, the data wasn’t immediately usable in its raw format. I had to extract and organize key fields such as the current stock price and market capitalization, from the JSON responses returned by Finnhub. This parsing step allowed me to structure the data into a clean format suitable for further calculations and analysis.


In [6]:
price = data2['c']
marketCap = data['marketCapitalization']
NumberOfShares = 'N/A'
print(price)
print(marketCap)

46.49
2786.542013634399


## 📊 Adding Stock Data to a Pandas DataFrame
Once I had the stock price and market capitalization parsed from the API responses, I added them to a Pandas DataFrame. This allowed me to structure the data in a clean, tabular format similar to a spreadsheet. Using the DataFrame made it easy to manage, process, and eventually export the data for further use, such as calculating share allocations and generating the final Excel output.

In [8]:
my_columns = ['Ticker', 'Stock Price','Market Capitalization', 'Number of Shares to Buy']
final_dataframe =  pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [ ]:
new_row = pd.Series([symbol, price, marketCap, NumberOfShares], index=final_dataframe.columns)
final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
final_dataframe

## 🔁 Looping Through the Tickers in My Stock List
Using the same logic I applied to a single stock, I looped through all 200 tickers in my list to fetch their data from the Finnhub API. For each symbol, I retrieved the stock price and market capitalization, then stored the results in my DataFrame. This approach allowed me to build a complete dataset with all the information needed to calculate equal-weight allocations across the selected companies.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks ['Ticker']:
    url = f"https://finnhub.io/api/v1/stock/profile2?symbol={stock}&token={api_key}"
    url2 = f"https://finnhub.io/api/v1/quote?symbol={stock}&token={api_key}"
    data = requests.get(url).json()
    data2 = requests.get(url2).json()
    new_row = pd.Series([stock, data2['c'], data['marketCapitalization'], NumberOfShares], index=final_dataframe.columns)
    final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)   

## 🚨 Using Batch API Calls to Improve Performance (Important)
Batch API calls are one of the most effective ways to improve performance in data-driven scripts, especially when working with large lists of stocks. Since HTTP requests can be time-consuming, batching them helps reduce the total number of API calls and speeds up data collection.

Initially, I considered optimizing my script using batch API functionality, as some other providers allow multiple tickers to be requested in a single call—dramatically reducing the number of required requests.

However, since I used the Finnhub API for this project, I wasn’t able to implement batching. Finnhub’s REST API currently doesn’t support batch requests for stock quotes or company profiles. As a result, I looped through each stock individually and made separate API calls for each ticker.

Even so, the script runs efficiently and remains well within Finnhub’s free-tier rate limits.

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,118.01,33851.993091,N/A
1,AAL,11.22,7439.299148,N/A
2,AAP,46.49,2786.542014,N/A
3,AAPL,205.17,3003295.926641,N/A
4,ABBV,185.62,322032.910987,N/A
...,...,...,...,...
193,HCA,383.1,91249.686303,N/A
194,HD,366.64,366869.742797,N/A
195,HES,138.54,42979.393071,N/A
196,HIG,126.87,35421.961037,N/A


In [ ]:
def chunks(lst, n):
    for i in range (0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
#We can't use batch in these url as Finnhub doesn't support multiple endpoints
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range (0, len(symbol_groups):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f""

## 🧮 Calculating the Number of Shares to Buy
At this point, the DataFrame contained the stock price and market capitalization for each company, but it didn’t yet include the number of shares to buy. To fix that, I calculated how many shares of each stock I could purchase by dividing an equal portion of the total portfolio value by the stock’s current price. This ensured an equal-weight allocation across all 200 companies in the portfolio.

In [11]:
portfolio_size =input('Enter the value of your portfolio')
try:
    val = float(portfolio_size)
    print (val)
except ValueError:
    print("Please enter an integer \n")
    portfolio_size =input('Enter the value of your portfolio')
    val = float(portfolio_size)

Enter the value of your portfolio 1000000


1000000.0


In [12]:
position_size = val/len(final_dataframe.index)
for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i, 'Stock Price'])
print (final_dataframe)
#number_of_apple_shares = position_size/12
#print(math.floor(number_of_apple_shares))

    Ticker Stock Price Market Capitalization Number of Shares to Buy
0        A      118.01          33851.993091                      42
1      AAL       11.22           7439.299148                     450
2      AAP       46.49           2786.542014                     108
3     AAPL      205.17        3003295.926641                      24
4     ABBV      185.62         322032.910987                      27
..     ...         ...                   ...                     ...
193    HCA       383.1          91249.686303                      13
194     HD      366.64         366869.742797                      13
195    HES      138.54          42979.393071                      36
196    HIG      126.87          35421.961037                      39
197    HII      241.46           9364.418906                      20

[198 rows x 4 columns]


In [13]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index= False)

C:\Users\sajid\AppData\Local\Temp\ipykernel_12832\3142007780.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_dataframe.to_excel(writer,'Recommended Trades', index= False)


### Creating the Formats I Used for the .xlsx File
To make the Excel output more readable and professional, I defined a few custom formats using XlsxWriter. These formats handled fonts, alignment, and number styles like currency and integers. I created four main formats for the final Excel document:

A string format for the stock tickers

A dollar format ($XX.XX) for stock prices

A dollar format with commas ($XX,XXX) for market capitalization

An integer format for the number of shares to purchase

These formats made the spreadsheet visually cleaner and easier to understand at a glance.

In [14]:
background_color = '#0a0a23'
font_color= '#ffffff'

string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})
dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})
integer_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

In [ ]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
#writer.close()

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [16]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format] 
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f"{column}:{column}", 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [18]:
writer.close()